In [1]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
import time

In [2]:
class Net2(nn.Module):
  def __init__(self):
      super(Net2, self).__init__()

      self.conv1 = nn.Conv2d(1,12,3, padding=1)
      self.bn_1 = nn.BatchNorm2d(num_features=12)
      self.do_1 = nn.Dropout(0.14)

      self.conv2 = nn.Conv2d(12, 16, 3)
      self.bn_2 = nn.BatchNorm2d(num_features=16)
      self.do_2 = nn.Dropout(0.14)
      
      self.conv3 = nn.Conv2d(16, 10, 1)
      self.bn_3 = nn.BatchNorm2d(num_features=10)
      
      self.pool1 = nn.MaxPool2d(2,2)

      self.conv4 = nn.Conv2d(10, 20, 3)
      self.bn_4 = nn.BatchNorm2d(num_features=20)
      self.do_4 = nn.Dropout(p=0.14)

      self.conv5 = nn.Conv2d(20, 16, 3)
      self.bn_5 = nn.BatchNorm2d(num_features=16)
      self.do_5 = nn.Dropout(0.14)

      self.conv6 = nn.Conv2d(16, 16, 3)
      self.bn_6 = nn.BatchNorm2d(num_features=16)
      self.do_6 = nn.Dropout(0.13)

      self.conv7 = nn.Conv2d(16, 10, 3)
      self.bn_7 = nn.BatchNorm2d(num_features=10)
      self.do_7 = nn.Dropout(0.14)


      # self.conv_1x1_1 = nn.Conv2d(20, 10, 1)
      # self.bn_1x1_1 = nn.BatchNorm2d(num_features=10)
      # self.do_4 = nn.Dropout(p=0.12)

      self.conv8 = nn.Conv2d(10, 10, 3)
      self.bn_8 = nn.BatchNorm2d(num_features=10)
      self.do_8 = nn.Dropout(0.12)

      self.conv9 = nn.Conv2d(10, 10, 3)
#       self.bn_9 = nn.BatchNorm2d(num_features=10)
#       self.do_9 = nn.Dropout(0.10)


#       self.conv10 = nn.Conv2d(10, 10, 3)
#       self.bn_10 = nn.BatchNorm2d(num_features=10)

      self.conv_1x1_2 = nn.Conv2d(10, 10, 1)


  def forward(self, x):
      x = self.conv1(x)
      x = F.relu(x)
      x = self.bn_1(x)
      x = self.do_1(x)

      x = self.conv2(x)
      x = F.relu(x)
      x = self.bn_2(x)
      x = self.do_2(x)


      x = self.conv3(x)
      x = F.relu(x)
      x = self.bn_3(x)

      x = self.pool1(x)
      
      x = self.conv4(x)
      x = F.relu(x)
      x = self.bn_4(x)
      x = self.do_4(x)

      # x = self.pool1(x)
      # x = self.do_2(x)

      x = self.conv5(x)
      x = F.relu(x)
      x = self.bn_5(x)
      x = self.do_5(x)

      x = self.conv6(x)
      x = F.relu(x)
      x = self.bn_6(x)
      x = self.do_6(x)

      x = self.conv7(x)
      x = F.relu(x)
      x = self.bn_7(x)
      x = self.do_7(x)
      
      x = self.conv8(x)
      x = F.relu(x)
      x = self.bn_8(x)
      x = self.do_8(x)

      x = self.conv9(x)
      # x = F.relu(x)
      # x = self.bn_9(x)
      # x = self.do_9(x)

      # x = self.conv10(x)
      # x = F.relu(x)
      # x = self.bn_10(x)

      x = self.conv_1x1_2(x)
      
      x = x.view(-1, 10)

      return F.log_softmax(x, dim=1)
      

In [3]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

use_cuda

True

In [4]:
model2 = Net2().to(device)

In [5]:
summary(model2, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
       BatchNorm2d-2           [-1, 12, 28, 28]              24
           Dropout-3           [-1, 12, 28, 28]               0
            Conv2d-4           [-1, 16, 26, 26]           1,744
       BatchNorm2d-5           [-1, 16, 26, 26]              32
           Dropout-6           [-1, 16, 26, 26]               0
            Conv2d-7           [-1, 10, 26, 26]             170
       BatchNorm2d-8           [-1, 10, 26, 26]              20
         MaxPool2d-9           [-1, 10, 13, 13]               0
           Conv2d-10           [-1, 20, 11, 11]           1,820
      BatchNorm2d-11           [-1, 20, 11, 11]              40
          Dropout-12           [-1, 20, 11, 11]               0
           Conv2d-13             [-1, 16, 9, 9]           2,896
      BatchNorm2d-14             [-1, 1

In [6]:
torch.manual_seed(100)
batch_size = 32

In [7]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [8]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [10]:
# torch.backends.cudnn.benchmark = True # found online, https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936 

In [11]:
def train(model, device, train_loader, optimizer, epochs, scheduler=None, test=False):
    model.train()
    for epoch in range(epochs):
      correct = 0
      train_loss = 0
      print(f"\n\nepoch: {epoch+1}")
      pbar = tqdm(train_loader)
      for batch_idx, (data, target) in enumerate(pbar):
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = F.nll_loss(output, target)
          train_loss += loss.item()
          loss.backward()
          optimizer.step()
          pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
          correct += pred.eq(target.view_as(pred)).sum().item()
          # correct += (output == target).float().sum()
          pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
      
      if scheduler:
          print(f'lr: {scheduler.get_last_lr()[0]}')
          scheduler.step()
      train_loss /= len(train_loader.dataset)
      print('\nTrain set:                              Accuracy: {}/{} ({:.6f}%)'.format(correct, len(train_loader.dataset), 100.0 * correct / len(train_loader.dataset)))

      if test:
          model.eval()
          test_loss = 0
          correct = 0
          with torch.no_grad():
              for data, target in test_loader:
                  data, target = data.to(device), target.to(device)
                  output = model(data)
                  test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
                  pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                  correct += pred.eq(target.view_as(pred)).sum().item()

          test_loss /= len(test_loader.dataset)

          print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.6f}%)\n'.format(
              test_loss, correct, len(test_loader.dataset),
              100. * correct / len(test_loader.dataset)))


# def test(model, device, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
#             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.6f}%)\n'.format(
#         test_loss, correct, len(test_loader.dataset),
#         100. * correct / len(test_loader.dataset)))

In [12]:
model2 = Net2().to(device)

In [13]:
optimizer = optim.SGD(params=model2.parameters(),lr = 0.01, momentum=0.9)

# optimizer = optim.Adam(params=model1.parameters())

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.13)

epochs = 10
print("\n")

train(model2, device, train_loader, optimizer, epochs, scheduler, test=True)

# test(model1, device, test_loader) # test after training

  0%|          | 0/1875 [00:00<?, ?it/s]





epoch: 1


loss=0.023644063621759415 batch_id=1874: 100%|██████████| 1875/1875 [00:28<00:00, 65.52it/s]

lr: 0.01

Train set:                              Accuracy: 55881/60000 (93.135000%)



  0%|          | 0/1875 [00:00<?, ?it/s]

...


  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0207, Accuracy: 9939/10000 (99.390000%)



epoch: 10


loss=0.0005642673349939287 batch_id=1874: 100%|██████████| 1875/1875 [00:28<00:00, 66.07it/s]

lr: 0.0013000000000000002

Train set:                              Accuracy: 59837/60000 (99.728333%)



Test set: Average loss: 0.0204, Accuracy: 9941/10000 (99.410000%)

